In [ ]:
import os
import torch
import pandas as pd
import mlflow

from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import torch.nn.functional as F
import pickle as pkl
import matplotlib.pyplot as plt


# Read data, convert to tensors

data_samples ="sonar.csv" 
data = pd.read_csv(data_samples, header=None)

X = data.iloc[:, 0:3].values
y = data.iloc[:, 3].values

# convert into PyTorch tensors
trainingSetX = torch.tensor(X, dtype=torch.float32)
trainingSetY = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

loader = DataLoader(list(zip(trainingSetX,trainingSetY)), shuffle=True)

# use Cuda Y/N 


device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)


print(f"Using {device} device")



#Neural Network construction


class NeuralNetwork(nn.Module):
    
    def __init__(self, inputs, outputs):
        
        super().__init__()
        
        # Simple linear connections
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, 36),
            nn.ReLU(),
            nn.Linear(36, 24),
            nn.ReLU(),
            nn.Linear(24, 12),
            nn.SiLU(),
            nn.Linear(12, outputs),
            # nn.ReLU()
        )
            
    def forward(self, x):
        
        return self.linear_relu_stack(x)

    
    

# Model Training: 
model = NeuralNetwork(3, 1)
print(model)


# Take binary cross entropy as loss function (one output interpreted as binary)
# lossFunction = nn.BCELoss()
lossFunction = nn.MSELoss()

learningRate = 0.00008

# Use stochastic gradient descent as optimizer, use weights and biases of model
gradientDescent = torch.optim.Adam(model.parameters(), lr=learningRate)

epochs = 100



losses = []





for epoch in range(epochs):
    
    
    added = False
    epochLoss = 0.0
    
    for x, y in loader:
    
        # Reset the gradient delta's (dw, db)
        gradientDescent.zero_grad()

        # Forward step
        yhat = model(x)

        # Compute loss
        loss = lossFunction(yhat, y)
        
        if not added:
            
            added = True
        
            # Keep track of loss
            epochLoss += loss.item()

        # Apply gradient descent (via backpropagation)
        loss.backward()

        # Use w += -step * dw * learnRate
        gradientDescent.step()
        
    losses.append(epochLoss / len(loader))
        
        
# ML flow       
        
sensor = "sonar"
hidden_layers = "36_24_12"
activation = "ReLU_ReLU_SiLU"

mlflow.set_experiment('NeuralNetwork')

# with mlflow.start_run(run_name=f'Neural net ({sensor}) | layers: {len(hidden_layers)}, epochs: {epochs}, lr: {lr}, batchsize: {batch_size}'):
with mlflow.start_run(run_name=f'NN({sensor}) {hidden_layers}, epochs: {epochs}, lr: {learningRate}'):


    mlflow.log_param('sensor', sensor)
    mlflow.log_param('data_samples', data_samples)
    
    mlflow.log_param('hidden_layers', hidden_layers)
    mlflow.log_param('activation', activation)

    mlflow.log_param('epochs', epochs)
    mlflow.log_param('lr', learningRate)
   # mlflow.log_param('batch_size', batch_size)

    mlflow.log_param('optimizer', gradientDescent)
    mlflow.log_param('loss_fn', lossFunction)
    mlflow.log_metric('epochLoss', losses[-1])

#save model

filehandler = open('SDCar_sonar.pth', 'wb')
torch.save(model.linear_relu_stack, filehandler)

# Compare

fig, ax = plt.subplots()
ax.set(xlabel='Epoch', ylabel='Cost', title="Training Cost")

plt.plot([x for x in range(epochs)], losses, 'red')
plt.show()

        